<a href="https://colab.research.google.com/github/DIPANJAN001/TalkJulia_with_Randy/blob/main/SolarPowerManagementSystemFinalllllll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [201]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import metrics

In [202]:
class MyEnv3(gym.Env):
    
      def __init__(self,seed=42):
        np.random.seed(seed)
        self.action_space = Discrete(4)
        self.observation_space=MultiDiscrete([150,130,50])
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        self.state,self.reward, self.done, self.info= None, None, None,None
        
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
      def _render(self):
        txt="state:"+str(self.state)+"reward"+str(self.reward)+"info:"+str(self.info)
        print(txt)

      def _states(self):
        observation_space=[]
        return observation_space
      def _reset(self):
        self.battery_cap=30
        self.state,self.reward,self.done,self.info=[30,0,0],0,False,{}
        return self.state

      def _step(self,action):
        done=False
        grid_export_=0
        grid_import_=0
        #load=np.random.randint(100,140)
        #pv=np.random.randint(80,130)
        battery_state=self.battery_cap
        reward=0
        data=[]
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-self.max_battery_discharge,0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        '''if(self.battery_cap<7.0):
              done=True'''
        #data.append([battery_state,load,pv,action,reward])
        #state.append(load)
        #state.append(pv)
        return reward,self.battery_cap, done,info

In [203]:
env=MyEnv3()

In [204]:
import time # Necessary to evaluate frugality
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import deque

In [205]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        #self.memory = deque(maxlen=2000)
        self.memory=[]
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(4,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        #minibatch = random.sample(self.memory,batch_size)
        minibatch = agent.memory[0:batch_size]
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict((next_state,))[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
            

In [206]:
agent = Agent(state_size=3,action_size=4)
done = False
state_size = 3
action_size =4
batch_size = 32

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [207]:
done=False
for e in range(100):
  score = 0
  counter=0
  #load=np.random.randint(100,140)
  #pv=np.random.randint(80,130)
  state=env._reset()
  done=False
  #state_=[]
  #state_.append(state[0])
  next_state=[]
  while not done:
    next_state=[]
    #state_.append(state[0])
    #load=np.random.randint(100,140)
    #pv=np.random.randint(80,130)
    #state_.append(load)
    #state_.append(pv)
    #print(state_)
    #state1=state_
    #action = env.action_space.sample()'''
    print(state)
    action=agent.act(state)
    #print(action)
    reward,observation2, done, info = env._step(action)

    load=np.random.randint(100,140)
    pv=np.random.randint(80,130)
    next_state.append(observation2)
    next_state.append(load)
    next_state.append(pv)
    #print(next_state)
    agent.remember(state,action,reward,next_state,done)
    score+=round(reward)
    state=next_state
    '''next_state.pop(0)
    next_state.pop(0)
    next_state.pop(0)'''
    #print(observation2)
    print(info)
    counter=counter+1
    if(counter==3000):
      done=True
    #state_[0]=observation2
    #agent.store_transition()
  print('episode:{} Profit:{}'.format(e,score))
  if len(agent.memory)>batch_size:
        agent.train(batch_size)

Streaming output truncated to the last 5000 lines.
{'grid_export': 0, 'grid_import': 0}
[0, 115, 106]
{'grid_export': 0, 'grid_import': 0}
[0, 124, 96]
{'grid_export': 0, 'grid_import': 0}
[0, 107, 94]
{'grid_export': 0, 'grid_import': 0}
[0, 106, 82]
{'grid_export': 0, 'grid_import': 9}
[0, 123, 82]
{'grid_export': 0, 'grid_import': 26}
[0, 114, 104]
{'grid_export': 0, 'grid_import': 0}
[0, 106, 96]
{'grid_export': 0, 'grid_import': 0}
[0, 119, 115]
{'grid_export': 0, 'grid_import': 0}
[0, 121, 81]
{'grid_export': 0, 'grid_import': 25}
[0, 110, 92]
{'grid_export': 0, 'grid_import': 0}
[0, 120, 97]
{'grid_export': 0, 'grid_import': 8}
[0, 117, 125]
{'grid_export': 0, 'grid_import': 0}
[8, 137, 111]
{'grid_export': 0, 'grid_import': 11}
[0, 124, 112]
{'grid_export': 0, 'grid_import': 0}
[0, 107, 90]
{'grid_export': 0, 'grid_import': 0}
[0, 123, 120]
{'grid_export': 0, 'grid_import': 0}
[0, 131, 94]
{'grid_export': 0, 'grid_import': 22}
[0, 121, 107]
{'grid_export': 0, 'grid_import': 0}


InvalidArgumentError: ignored

In [208]:
agent.memory

[([30, 0, 0], 0, 0, [0, 128, 94], False),
 ([0, 128, 94], 0, 0, [0, 120, 118], False),
 ([0, 120, 118], 2, 0, [0, 122, 90], False),
 ([0, 122, 90], 1, 0, [0, 135, 119], False),
 ([0, 135, 119], 3, -10, [0, 121, 81], False),
 ([0, 121, 81], 1, 0, [0, 129, 117], False),
 ([0, 129, 117], 1, 0, [0, 120, 112], False),
 ([0, 120, 112], 3, 0, [0, 121, 123], False),
 ([0, 121, 123], 1, 0, [2, 126, 121], False),
 ([2, 126, 121], 1, 0, [0, 115, 94], False),
 ([0, 115, 94], 0, 0, [0, 102, 116], False),
 ([0, 102, 116], 3, 0, [14, 120, 88], False),
 ([14, 120, 88], 3, -170, [0, 103, 104], False),
 ([0, 103, 104], 1, 0, [1, 108, 105], False),
 ([1, 108, 105], 0, 0, [0, 119, 107], False),
 ([0, 119, 107], 3, 0, [0, 106, 123], False),
 ([0, 106, 123], 2, 0, [0, 134, 93], False),
 ([0, 134, 93], 1, 0, [0, 139, 83], False),
 ([0, 139, 83], 1, 0, [0, 105, 121], False),
 ([0, 105, 121], 0, 48, [10, 128, 97], False),
 ([10, 128, 97], 0, 0, [0, 133, 89], False),
 ([0, 133, 89], 0, 0, [0, 113, 110], False),

In [209]:
minibatch = agent.memory[0:batch_size]

In [210]:
minibatch

[([30, 0, 0], 0, 0, [0, 128, 94], False),
 ([0, 128, 94], 0, 0, [0, 120, 118], False),
 ([0, 120, 118], 2, 0, [0, 122, 90], False),
 ([0, 122, 90], 1, 0, [0, 135, 119], False),
 ([0, 135, 119], 3, -10, [0, 121, 81], False),
 ([0, 121, 81], 1, 0, [0, 129, 117], False),
 ([0, 129, 117], 1, 0, [0, 120, 112], False),
 ([0, 120, 112], 3, 0, [0, 121, 123], False),
 ([0, 121, 123], 1, 0, [2, 126, 121], False),
 ([2, 126, 121], 1, 0, [0, 115, 94], False),
 ([0, 115, 94], 0, 0, [0, 102, 116], False),
 ([0, 102, 116], 3, 0, [14, 120, 88], False),
 ([14, 120, 88], 3, -170, [0, 103, 104], False),
 ([0, 103, 104], 1, 0, [1, 108, 105], False),
 ([1, 108, 105], 0, 0, [0, 119, 107], False),
 ([0, 119, 107], 3, 0, [0, 106, 123], False),
 ([0, 106, 123], 2, 0, [0, 134, 93], False),
 ([0, 134, 93], 1, 0, [0, 139, 83], False),
 ([0, 139, 83], 1, 0, [0, 105, 121], False),
 ([0, 105, 121], 0, 48, [10, 128, 97], False),
 ([10, 128, 97], 0, 0, [0, 133, 89], False),
 ([0, 133, 89], 0, 0, [0, 113, 110], False),

In [211]:
for state,action,reward,next_state,done in minibatch:
    print(state,next_state,reward)

[30, 0, 0] [0, 128, 94] 0
[0, 128, 94] [0, 120, 118] 0
[0, 120, 118] [0, 122, 90] 0
[0, 122, 90] [0, 135, 119] 0
[0, 135, 119] [0, 121, 81] -10
[0, 121, 81] [0, 129, 117] 0
[0, 129, 117] [0, 120, 112] 0
[0, 120, 112] [0, 121, 123] 0
[0, 121, 123] [2, 126, 121] 0
[2, 126, 121] [0, 115, 94] 0
[0, 115, 94] [0, 102, 116] 0
[0, 102, 116] [14, 120, 88] 0
[14, 120, 88] [0, 103, 104] -170
[0, 103, 104] [1, 108, 105] 0
[1, 108, 105] [0, 119, 107] 0
[0, 119, 107] [0, 106, 123] 0
[0, 106, 123] [0, 134, 93] 0
[0, 134, 93] [0, 139, 83] 0
[0, 139, 83] [0, 105, 121] 0
[0, 105, 121] [10, 128, 97] 48
[10, 128, 97] [0, 133, 89] 0
[0, 133, 89] [0, 113, 110] 0
[0, 113, 110] [0, 107, 93] 0
[0, 107, 93] [0, 139, 100] 0
[0, 139, 100] [0, 117, 126] -240
[0, 117, 126] [0, 125, 104] 0
[0, 125, 104] [0, 128, 94] 0
[0, 128, 94] [0, 124, 86] 0
[0, 124, 86] [0, 100, 123] -230
[0, 100, 123] [23, 110, 96] 0
[23, 110, 96] [9, 134, 112] 0
[9, 134, 112] [0, 138, 120] -70


In [213]:
minibatch = agent.memory[0:batch_size]
for state,action,reward,next_state,done in minibatch:
            
    if not done: #boolean 
          target = reward + agent.gamma*np.amax(agent.model.predict((next_state,))[0])
    else:
          target = reward
          target_f = agent.model.predict(state)
          target_f[0][action] = target
          agent.model.fit(state,target_f,epochs=1,verbose=0)

1/1 [==============================] - 0s 34ms/step


In [ ]:
agent.model.predict(([0, 134, 112],))